In [1]:
!pip install langchain_groq
!pip install chromadb
!pip install -qU langchain_community beautifulsoup4

In [2]:
!pip install --upgrade langchain langchain-community langchain-openai


In [3]:
!pip install langchain-groq


In [4]:
!pip install --upgrade langchain


In [5]:
!pip install langchain_groq # Make sure the langchain_groq package is installed and up-to-date.



In [6]:
from langchain_groq import ChatGroq

In [7]:
!pip install --upgrade langchain langchain-groq chromadb pandas requests

In [8]:
import os
import json
import requests
import pandas as pd
import chromadb
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.document_loaders import WebBaseLoader


In [9]:
def setup_vector_db():
    os.makedirs("vectorstore", exist_ok=True)
    client = chromadb.PersistentClient("vectorstore")
    return client.get_or_create_collection("cold_email_db")

In [10]:
# Set API Key
os.environ["GROQ_API_KEY"] = "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"

# Load portfolio data
if __name__ == "__main__":
    portfolio_df = pd.read_csv("/content/my_portfolio.csv")

def extract_job_details(page_data):
    prompt = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}

        ### INSTRUCTION:
        Extract job details in JSON format with the following structure:
        {{
            "role": "Job Title",
            "experience": "Experience required",
            "skills": ["Skill1", "Skill2", "Skill3"],
            "description": "Job Description"
        }}

        **IMPORTANT:**
        - Return **only** valid JSON output. No extra text.
        - Do **not** change the field names.
        - Do **not** return markdown (```json) or any other format.

        OUTPUT:
        """
    )

    llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7, api_key=os.getenv("GROQ_API_KEY"))
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(page_data=page_data)

    try:
        job_details = json.loads(response.strip())
        return job_details
    except json.JSONDecodeError:
        print(f"❌ Error parsing job details JSON: {response}")
        return None

def generate_cold_email(job_details, portfolio):
    prompt = PromptTemplate.from_template(
        """
        You are a professional job applicant. Based on the job details and your portfolio, generate a compelling cold email:

        ### JOB DETAILS:
        {job_details}

        ### YOUR PORTFOLIO:
        {portfolio}

        **INSTRUCTION:**
        - Keep it short and professional.
        - Highlight relevant experience.
        - End with a strong call to action.
        - Use a natural and polite tone.

        OUTPUT:
        """
    )

    llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7, api_key=os.getenv("GROQ_API_KEY"))
    chain = LLMChain(llm=llm, prompt=prompt)
    email = chain.run(job_details=job_details, portfolio=portfolio)
    return email.strip()

# Process job postings
job_postings = ["https://example.com/react-portfolio", "https://example.com/vue-portfolio"]
email_results = []

for job_url in job_postings:
    print(f"Scraping job posting: {job_url}")
    job_text = "Job description for testing"

    job_info = extract_job_details(job_text)

    if job_info:
        portfolio_info = portfolio_df.to_dict(orient='records')[0]
        cold_email = generate_cold_email(job_info, portfolio_info)
        email_results.append({"job": job_info, "email": cold_email})

        print("✅ Cold Email Generated:")
        print(cold_email)
    else:
        print("❌ Failed to extract job details.")

# Save results to JSON file
with open("generated_emails.json", "w") as f:
    json.dump(email_results, f, indent=4)

print("📩 Cold emails saved in generated_emails.json ✅")


Scraping job posting: https://example.com/react-portfolio


<ipython-input-10-625b1f0468c0>:33: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
<ipython-input-10-625b1f0468c0>:34: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(page_data=page_data)


✅ Cold Email Generated:
Subject: Application for Testing Job Opportunity

Dear Hiring Manager,

I came across the Testing Job role at your organization and was impressed by the job description. With 3+ years of experience in Testing and Quality Assurance, I believe I would be a great fit for this position. My expertise in Automation has allowed me to streamline testing processes, ensuring high-quality deliverables.

Although my background is in development with a tech stack comprising React, Node.js, and MongoDB (portfolio: https://example.com/react-portfolio), my passion for testing has driven me to acquire skills that align with your requirements.

I would appreciate the opportunity to discuss how my skills can contribute to your team's success. Please let's schedule a call to explore this further.

Best regards,
[Your Name]
Scraping job posting: https://example.com/vue-portfolio
✅ Cold Email Generated:
Subject: Expressed Interest in Testing Job Opportunity

Dear Hiring Manager,

I c